In [ ]:
import sys
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
sys.path.append('/content/RQC-Robotics-tactile_sensor')
import sensor_lib as sl
import os
dir='/content/RQC-Robotics-tactile_sensor/'
os.mkdir(dir+'generated_gauses')
os.mkdir(dir+'models_weights')
os.mkdir(dir+'resalts')

In [ ]:
n_gauses=10
n_pic=1000
n_fibers=4
n_rot=1
lerning_rate=1e-4
n_epochs=30

In [ ]:
sl.generate_pressure_map(
    n_pic, n_fibers, x= 97, y = 97,
    part=dir+'generated_gauses/fresh_gauss'+str(n_gauses)+'_'+str(n_pic)+'.npy')

In [ ]:
input, output, input_test, output_test = sl.sim_on_gpu(
    dir+'generated_gauses/fresh_gauss'+str(n_gauses)+'_'+str(n_pic)+'.npy',
    n_random_rot=n_rot, n_angles=n_fibers,
    batch_size_preproc=128*8
    )
input_shape = input.shape
output_shape = output.shape
dataset_b, dataset = sl.prepare_dataset_for_train(
    input, output, batch_size_fit_model=1024*2
    )
dataset_test_b, dataset_test = sl.prepare_dataset_for_train(
    input_test, output_test,batch_size_fit_model=1024*2
    )

In [ ]:
import tensorflow as tf

In [ ]:
model = sl.SensorNN4S(input_shape[1:3], output_shape[1:3])
model.build(input_shape)
model.summary()
model.compile(
    optimizer=tf.keras.optimizers.Adam(lerning_rate),
    loss=tf.keras.losses.MeanSquaredError(),
    metrics=['accuracy']
    )

In [ ]:
model.fit(dataset_b, epochs = n_epochs, verbose=1)

In [ ]:
model.save('models_weights/model4S_'+str(n_fibers)+'fi.nn')

In [ ]:
model.evaluate(dataset_test_b)

In [ ]:
predictions = model.predict(dataset_test_b)
predictions.shape

In [ ]:
with open(dir+'resalts/pred_4S_'+str(n_gauses)+'g_'+str(n_fibers)+'fi', 'wb') as f:
    np.save(f, predictions)
    np.save(f, output_test)

In [ ]:
N = 4 #28 26 24 23 21 20
plt.imshow(predictions[N])
plt.show()
plt.imshow(output_test[N])
plt.show()